In [1]:
import cudaq
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [5]:
# ----------------------------
# Quantum kernel
# ----------------------------
@cudaq.kernel
def qnn_kernel(x: list[float], thetas: list[float]):
    q = cudaq.qvector(len(x))
    # Data encoding
    for i in range(len(x)):
        ry(x[i], q[i])
    # Variational rotations
    for i in range(len(x)):
        rz(thetas[i], q[i])
    # Entanglement
    for i in range(len(x) - 1):
        cx(q[i], q[i+1])

# Observable
observable = sum([cudaq.spin.z(i) for i in range(2)])  # adjust to n_qubits

# ----------------------------
# Parameter-shift wrapper
# ----------------------------
def quantum_expectation(x, thetas):
    """Compute expectation value for given input and parameters."""
    return cudaq.observe(qnn_kernel, observable, list(x), list(thetas)).expectation()

def parameter_shift_gradient(x, thetas, shift=np.pi/2):
    grads = np.zeros_like(thetas)
    for i in range(len(thetas)):
        shifted = thetas.copy()
        shifted[i] += shift
        forward = quantum_expectation(x, shifted)

        shifted[i] -= 2 * shift
        backward = quantum_expectation(x, shifted)

        grads[i] = 0.5 * (forward - backward)
    return grads

# ----------------------------
# Torch Module with custom backward
# ----------------------------
class QNNFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, thetas):
        x_np = x.detach().cpu().numpy()
        thetas_np = thetas.detach().cpu().numpy()
        out_vals = [quantum_expectation(row[:len(thetas_np)], thetas_np) for row in x_np]
        ctx.save_for_backward(x, thetas)
        return torch.tensor(out_vals, dtype=torch.float32, device=x.device).view(-1, 1)

    @staticmethod
    def backward(ctx, grad_output):
        x, thetas = ctx.saved_tensors
        x_np = x.detach().cpu().numpy()
        thetas_np = thetas.detach().cpu().numpy()

        grad_thetas = np.zeros_like(thetas_np)
        for row in x_np:
            grad_thetas += parameter_shift_gradient(row[:len(thetas_np)], thetas_np)
        grad_thetas /= len(x_np)

        return None, torch.tensor(grad_thetas, dtype=torch.float32, device=thetas.device)

class QNNRegressor(nn.Module):
    def __init__(self, n_qubits=4):
        super().__init__()
        self.params = nn.Parameter(torch.randn(n_qubits))

    def forward(self, x):
        return QNNFunction.apply(x, self.params)


In [6]:
# ----------------------------
# Load dataset
# ----------------------------
df = pd.read_csv("../../data/data.csv")
num_cols = df.select_dtypes(include=[np.number]).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].mean())

drop_cols = ["ID","Unnamed: 0","e_Sint"]
features = [c for c in df.columns if c not in drop_cols and c != "Sint"]

X = df[features].values.astype(np.float32)
y = df["Sint"].values.astype(np.float32).reshape(-1,1)

scalerX, scalery = StandardScaler(), StandardScaler()
X = scalerX.fit_transform(X)
y = scalery.fit_transform(y)

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# ----------------------------
# Train
# ----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = QNNRegressor(n_qubits=2).to(device)
opt = optim.Adam(model.parameters(), lr=0.1)
loss_fn = nn.MSELoss()

for epoch in range(5):
    model.train()
    opt.zero_grad()
    preds = model(X_train.to(device))
    loss = loss_fn(preds, y_train.to(device))
    loss.backward()
    opt.step()
    print(f"Epoch {epoch+1}, Loss={loss.item():.6f}")

Epoch 1, Loss=2.238206
Epoch 2, Loss=2.238206
Epoch 3, Loss=2.238206
Epoch 4, Loss=2.238206
Epoch 5, Loss=2.238206


In [7]:
# ----------------------------
# Evaluate
# ----------------------------
model.eval()
with torch.no_grad():
    y_pred = model(X_test.to(device)).cpu().numpy()
    y_true = y_test.cpu().numpy()

mse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
r2  = r2_score(y_true, y_pred)

print("\n--- Sint Metrics ---")
print("MSE :", mse)
print("MAE :", mae)
print("R²  :", r2)


--- Sint Metrics ---
MSE : 2.4356324672698975
MAE : 1.1046563386917114
R²  : -1.3300580978393555
